In [1]:
import torch
import torchtext
from torchtext import data
from torchtext.vocab import GloVe
from torch import nn,optim
import torch.nn.functional as f

In [14]:
class WikiQA_dataset():
    def __init__(self,embedding_dim,batch_size):
        self.RAW = data.RawField()
        self.TEXT = data.Field(batch_first=True,
                               lower=True)
        self.LABEL = data.Field(sequential=False,
                                unk_token=None)
    
        
        self.train, self.dev, self.test = data.TabularDataset.splits(
            path='.',
            train='WikiQA-train.tsv',
            validation='WikiQA-dev.tsv',
            test='WikiQA-test.tsv',
            format='tsv',
            fields=[('qid', self.RAW),
                    ('question', self.TEXT),
                    ('docid', self.RAW),
                    ('doctitle', self.RAW),
                    ('sentenceid',self.RAW),
                    ('sentence',self.TEXT),
                    ('label',self.LABEL)])
        
        self.TEXT.build_vocab(self.train, self.dev, self.test, \
                              vectors=GloVe(name='6B', dim=embedding_dim), \
                              unk_init=torch.zeros((1, embedding_dim)).uniform_(-0.25, 0.25))
        
        self.LABEL.build_vocab(self.train)
        
        self.train_iter, self.dev_iter, self.test_iter = \
            data.Iterator.splits((self.train, self.dev, self.test),
                                 batch_sizes=[batch_size] * 3,
                                 shuffle=True)
        
        
        

In [15]:
d = WikiQA_dataset(20,20)